# 5章 誤差逆伝播法

数値微分はシンプルで、実装は 簡単でしたが、計算に時間がかかるという難点があります。本章では、重みパラメー タの勾配の計算を効率良く行う手法である「誤差逆伝播法」について学びます。

## 5.1 計算グラフ
計算グラフとは、計算の過程をグラフによって表したものです。ここで言うグラフとは、データ構造としてのグラフであり、複数のノードとエッジによって表現されます(ノード間を結ぶ直線を「エッジ」と言います)。

## 5.2 連鎖律
これまで行ってきた計算グラフの順伝播は、計算の結果を順方向に――左から右方 向へ――伝達しました。そのとき行った計算は、日頃行う計算であるため自然に感じ られたことでしょう。一方、逆方向の伝播では「局所的な微分」を、順方向とは逆向 きに――右から左方向へ――伝達していきます(これは最初はとまどうかもしれませ ん)。なお、この「局所的な微分」を伝達する原理は、連鎖律(chain rule)によるも のです。ここでは連鎖律について説明し、それが計算グラフ上での逆伝播に対応する ことを明らかにします。

### 5.4.1 乗算レイヤの実装
レイヤは、forward() と backward() という共通のメソッド(インタフェース)を 持つように実装します。forward() は順伝播、backward() は逆伝播に対応します。

In [7]:
import numpy as np

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

__init__() では、インスタンス変数である x と y の初期化を行いますが、これ らは、順伝播時の入力値を保持するために用います。<br>
forward() では、x、y の 2 つ の引数を受け取り、それらを乗算して出力します。<br>
一方、backward() では、上流から伝わってきた微分(dout)に対して、順伝播の“ひっくり返した値”を乗算して下 流に流します。

In [2]:
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
# bacward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


ここで、backward() の呼び出す順番は、forward() のときと逆の順番で行いま す。また、backward() の引数は、「順伝播の際の出力変数に対する微分」を入力する ことに注意しましょう。たとえば、mul_apple_layer という乗算レイヤは、順伝播 時に apple_price を出力しますが、逆伝播時には、apple_price の微分値である dapple_price を引数に設定します。

In [4]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

加算レイヤでは特に初期化は必要ないので、__init__() では何も行いません (pass という記述は「何も行わない」という命令です)。加算レイヤの forward() では、2 つの引数 x、y を受け取り、それらを加算して出力します。backward() では上流から伝わってきた微分(dout)を、そのまま下流に流すだけです。

In [5]:
# リンゴ 2 個とみかん 3 個の買い物

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dall_price)

print(price)
print(dapple, dapple_num, dorange, dapple_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 110.00000000000001 650


この実装は多少長くなりましたが、一つひとつの命令は単純です。必要なレイヤを 作成し、順伝播のメソッド forward() を適切な順番で呼び出します。<br>
そして、順伝播と逆の順番で逆伝播のメソッド backward() を呼び出すと、求めたい微分が得られます。<br>
このように、計算グラフにおけるレイヤの実装――ここでは乗算と加算――は簡単 に行うことができ、それらを使えば複雑な微分の計算を求めることができます。続いて、ニューラルネットワークで使われるレイヤを実装していきます。

## 5.5 活性化関数レイヤの実装

In [6]:
# ReLU レイヤの実装
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

Relu クラスは、インスタンス変数として mask という変数を持ちます。この mask 変数は True/False からなる NumPy 配列で、順伝播の入力である x の要素で 0 以 下の場所を True、それ以外(0 より大きい要素)を False として保持します。

In [9]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [10]:
# Sigmoid レイヤ
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

この実装では、順伝播時に出力をインスタンス変数の out に保持しておきます。そ して、逆伝播時に、その out 変数を使って計算を行います。

## 5.6 Affine / Softmax レイヤの実装

### 5.6.2：Affineレイヤの実装【ゼロつく1のノート(実装)】
(https://www.anarchive-beta.com/entry/2020/08/02/180000)
Affineレイヤに関する解説

In [11]:
class Affine:
    def __init__(self, W, b):
        self.W = W # 重み
        self.b = b # バイアス
        self.x = None # 順伝播の入力
        self.dW = None # 重みの勾配
        self.db = None # バイアスの勾配

    def forward(self, x):
        self.x = x # 順伝播の入力を保存
        out = np.dot(x, self.W) + self.b # 重み付き和の計算

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T) # 順伝播の入力の勾配
        self.dW = np.dot(self.x.T, dout) # 重みの勾配
        self.db = np.sum(dout, axis=0) # バイアスの勾配

        return dx

In [12]:
import sys, os
sys.path.append(os.pardir)
from func.function import *

In [13]:
# Softmax-with-Loss レイヤの実装
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

この実装では、「3.5.2 ソフトマックス関数の実装上の注意」と「4.2.4 [バッ チ対応版]交差エントロピー誤差の実装」で実装した関数―― softmax() と cross_entropy_error()――を利用しています。そのため、ここでの実装はと ても簡単です。また、逆伝播の際には、伝播する値をバッチの個数(batch_size) で割ることで、データ 1 個あたりの誤差が前レイヤへ伝播する点に注意しましょう。

## 5.7 誤差逆伝播法の実装

In [2]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from func.function import *
from func.layer import *
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * \
            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

In [3]:
# 誤差逆伝播法の勾配確認
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:0.0009474537527150702
b1:0.006001038872339556
W2:0.01466766668906883
b2:0.30633156198179856


本章では、視覚的に計算の過程を表す計算グラフという方法を学びました。この 計算グラフを用いて、ニューラルネットワークで行う誤差逆伝播法を説明し、また、 ニューラルネットワークで行う処理をレイヤという単位で実装しました。たとえば、 ReLU レイヤや Softmax-with-Loss レイヤ、Affine レイヤや Softmax レイヤなど です。これらのレイヤには、forward と backward というメソッドが実装されてお り、データを順方向と逆方向に伝播することで、重みパラメータの勾配を効率的に求 めることができます。このレイヤによるモジュール化によって、ニューラルネット ワークでは、レイヤを自由に組み合わせることができ、自分の好きなネットワークを 簡単に作ることができるようになるのです。

● 計算グラフを用いれば、計算過程を視覚的に把握することができる。<br>
● 計算グラフのノードは局所的な計算によって構成される。局所的な計算が
全体の計算を構成する。<br>
● 計算グラフの順伝播は、通常の計算を行う。一方、計算グラフの逆伝播に
  よって、各ノードの微分を求めることができる。<br>
● ニューラルネットワークの構成要素をレイヤとして実装することで、勾配
の計算を効率的に求めることができる(誤差逆伝播法)。<br>
● 数値微分と誤差逆伝播法の結果を比較することで、誤差逆伝播法の実装に
誤りがないことを確認できる(勾配確認)。<br>